# How is occupancy computed?

In order to compute aggregate summary statistics for occupancy by time of day and day of week, we first need to determine how many patients (entities, in general) are present during each time bin of each date over the analysis range. In our Short Stay Unit (SSU) example, we have been using an *analysis date range* of 2024-01-02 through 2024-09-30. Let's use hourly time bins. This leads to 6552 hourly datetime bins as shown below. Note that we are using a [24-hour clock](https://simple.wikipedia.org/wiki/24-hour_clock).



In [2]:
import pandas as pd

In [4]:
bydatetime_df = pd.read_csv('output/cli_demo_ssu_60_bydatetime_datetime.csv')
bydatetime_df['datetime']

0       2024-01-02 00:00:00
1       2024-01-02 01:00:00
2       2024-01-02 02:00:00
3       2024-01-02 03:00:00
4       2024-01-02 04:00:00
               ...         
6547    2024-09-30 19:00:00
6548    2024-09-30 20:00:00
6549    2024-09-30 21:00:00
6550    2024-09-30 22:00:00
6551    2024-09-30 23:00:00
Name: datetime, Length: 6552, dtype: object

Assume the very first patient arrives at 06:15 on 2024-01-02 and departs at 09:36 the same day. For the time bins starting at 07:00 and 08:00, the patient is in the unit for the entire time bin. However, for the *arrival bin*, 06:00, the patient is only present for 45 minutes. Similarly, for the *departure bin*, the patient is in the unit for 36 minutes. 

During the hillmaking process, occupancy contributions by datetime bin for each patient are computed and are accumulated in NumPy arrays. Eventually these arrays are converted to a pandas `DataFrame` that we refer to as the *bydatetime* table or dataframe.


## What about the boundaries of the analysis date range?

As long as the records are in the stops dataframe, hillmaker will account for patients who might have arrived before but discharged after the start date. In our SSU example, the start date was 2024-01-02 because we wanted to ignore the impact of the January 1 holiday. However, records from 2024-01-01 are in the stops dataframe. 

In [9]:
ssu_stopdata = './data/ssu_2024.csv'
stops_df = pd.read_csv(ssu_stopdata, parse_dates=['InRoomTS','OutRoomTS'])
stops_df[stops_df['InRoomTS'] < pd.Timestamp('2024-01-02')]

,PatID,InRoomTS,OutRoomTS,PatType,LOS_hours
0,1,2024-01-01 07:44:00,2024-01-01 09:20:00,IVT,1.600000
1,2,2024-01-01 08:28:00,2024-01-01 11:13:00,IVT,2.750000
2,3,2024-01-01 11:44:00,2024-01-01 12:48:00,MYE,1.066667
3,4,2024-01-01 11:51:00,2024-01-01 21:10:00,CAT,9.316667
4,5,2024-01-01 12:10:00,2024-01-01 12:57:00,IVT,0.783333
5,6,2024-01-01 14:16:00,2024-01-01 17:35:00,IVT,3.316667
6,7,2024-01-01 14:40:00,2024-01-01 17:24:00,IVT,2.733333
7,8,2024-01-01 17:25:00,2024-01-02 01:53:00,CAT,8.466667


Notice that the last patient who arrived on 2024-01-01 wasn't discharged until 2024-01-02 01:53. If we look at the bydatetime table, we can see the occupanc contributions of this patient.

In [11]:
bydatetime_df.head(3)

,datetime,arrivals,departures,occupancy,dow_name,bin_of_day_str,day_of_week,bin_of_day,bin_of_week
0,2024-01-02 00:00:00,0.0,0.0,1.000000,Tue,00:00,1,0,24
1,2024-01-02 01:00:00,0.0,1.0,0.883333,Tue,01:00,1,1,25
2,2024-01-02 02:00:00,0.0,0.0,0.000000,Tue,02:00,1,2,26


We see that:

- there is one patient in the system from 12a-1a. This patient arrived on 2024-01-01 and had not yet been discharged as of midnight on 2024-01-02. 
- between 1a-2a (at 01:53), this patient was discharged. The occupancy value of 0.883333 for the 1a-2a time bin means that the patient was discharged after spending approximately $88\%$ ($53/60$ minutes) of the 1a-2a period in the SSU. 

Similarly, those patients who arrive during the analysis date range but are discharged after the end date, are included by hillmaker for the time spent in the system duing the analysis date range.

# Other options for computing occupancy 

This way of computing occupancy is the default way that hillmaker computes it. However, if you decide you want to use an occupancy contribution of 1.0 even for time bins in which the patient was only present for a fraction of that time, you can use do this through the `edge_bins` parameter. From the help for the `make_hills()` function:

```
edge_bins: int, default 1
    Occupancy contribution method for arrival and departure bins. 1=fractional, 2=whole bin
```


```{warning}
You can grossly overestimate occupancy if you use ... TODO: SHOULD WE EVEN HAVE THIS OPTION ANYMORE? NOW THAT WE ADDED THE ABILITY TO COMPUTE THE BYDATETIME TABLE AT HIGHER RESOLUTIONS THAN bin_size_minutes, I'M NOT SURE THIS MAKES SENSE ANYMORE
```